In [1]:
old_params = 0
old_image = 0
old_img = 0
contribution_map = 0

def remove_module_prefix(state_dict):
    new_state_dict = {}
    for k, v in state_dict.items():
        if k.startswith('module.'):
            new_key = k[7:] 
            new_state_dict[new_key] = v
        else:
            new_state_dict[k] = v
    return new_state_dict

In [2]:
import os
import random
import sys
import pickle
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet50
from torchvision.models import ResNet50_Weights
from PIL import Image
import pydicom
from sklearn.model_selection import KFold
from bcosconv2d import NormedConv2d
from torchvision.transforms import functional as TF
from torchvision.transforms import ToPILImage

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

model_path = r"C:\Users\Admin\Downloads\pneumonia_detection_model_bcos_trans_bestf1_1_20.pth"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\splits\splits_balanced.pkl"
csv_path = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
image_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
data = pd.read_csv(csv_path)

splits = 0
with open(splits_path, 'rb') as f:
    splits = pickle.load(f)
    
first_split = splits[0]
val_fifth_split = first_split[1]
print(val_fifth_split)

# LOAD MODEL
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)    
model.fc.linear = NormedConv2d(2048, 1, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network
state_dict = torch.load(model_path, map_location=device)
#fixed_dict = remove_module_prefix(state_dict)
model.load_state_dict(state_dict)
model = model.to(device)


model.eval()

count = 0
for index in val_fifth_split[:10]:
    count +=1
    row = data.iloc[index]
    image_path = os.path.join(image_folder, f"{row['patientId']}.dcm")
    label = row['Target']
        
    dicom = pydicom.dcmread(image_path)
    image = dicom.pixel_array
    
    image = Image.fromarray(image).convert("RGB")
    tensor_image = TF.to_tensor(image)
    img_new = model.transform(image)
    img_new = img_new[None]
    img_new = img_new.to(device)

    
    expl = model.explain(img_new)
    print(count)
    
    #print(expl)
    #print(expl["contribution_map"] == contribution_map)
    #contribution_map = expl["contribution_map"]
    prediction = expl["prediction"]
    if (prediction == 0):
        print("Prediction:", expl["prediction"])
        plt.imshow(expl["explanation"])
        plt.show()
    
    del(img_new)
    torch.cuda.empty_cache()





[    4     9    12 ... 30214 30221 30223]


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


RuntimeError: Error(s) in loading state_dict for BcosResNet:
	Missing key(s) in state_dict: "conv1.linear.weight", "layer2.0.conv2.linear.weight", "layer2.0.downsample.0.linear.weight", "layer3.0.conv2.linear.weight", "layer3.0.downsample.0.linear.weight", "layer4.0.conv2.linear.weight", "layer4.0.downsample.0.linear.weight". 
	Unexpected key(s) in state_dict: "conv1.conv.linear.weight", "conv1.blurpool.filt", "layer2.0.conv2.conv.linear.weight", "layer2.0.conv2.blurpool.filt", "layer2.0.downsample.0.conv.linear.weight", "layer2.0.downsample.0.blurpool.filt", "layer3.0.conv2.conv.linear.weight", "layer3.0.conv2.blurpool.filt", "layer3.0.downsample.0.conv.linear.weight", "layer3.0.downsample.0.blurpool.filt", "layer4.0.conv2.conv.linear.weight", "layer4.0.conv2.blurpool.filt", "layer4.0.downsample.0.conv.linear.weight", "layer4.0.downsample.0.blurpool.filt". 
	size mismatch for fc.linear.weight: copying a param with shape torch.Size([2, 2048, 1, 1]) from checkpoint, the shape in current

In [1]:
model.eval()
count = 0
for index in val_fifth_split[:5]:
    count +=1
    row = data.iloc[index]
    image_path = os.path.join(image_folder, f"{row['patientId']}.dcm")
    label = row['Target']
        
    dicom = pydicom.dcmread(image_path)
    image = dicom.pixel_array
    
    image = Image.fromarray(image).convert("RGB")
    tensor_image = TF.to_tensor(image)
    six_channel_image = torch.cat([tensor_image, 1-tensor_image], dim=0) # according
    six_channel_image = six_channel_image[None]
    six_channel_image = six_channel_image.to(device)
    
    expl = model.explain(six_channel_image)
    print(count)
    
    #print(expl)
    #print(expl["contribution_map"] == contribution_map)
    #contribution_map = expl["contribution_map"]
    prediction = expl["prediction"]
    if (prediction == 0):
        print("Prediction:", expl["prediction"])
        plt.imshow(expl["explanation"])
        plt.show()
    
    del(six_channel_image)
    torch.cuda.empty_cache()

NameError: name 'model' is not defined

In [ ]:
import os
from IPython.display import display
from PIL import Image

def display_images_in_notebook(directory):
    # Supported image file extensions
    image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp'}

    try:
        # Loop through all files in the directory
        for filename in os.listdir(directory):
            # Get the file extension in lowercase
            _, extension = os.path.splitext(filename)
            extension = extension.lower()

            # Check if the file is an image
            if extension in image_extensions:
                image_path = os.path.join(directory, filename)
                
                # Open the image
                img = Image.open(image_path)
                
                # Display the image inline in the notebook
                print(f"Displaying: {filename}")
                display(img)
    except FileNotFoundError:
        print(f"The directory '{directory}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Specify the directory
image_directory = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images\00c0b293-48e7-4e16-ac76-9269ba535a62.dcm"

# Display all images in the notebook
display_images_in_notebook(image_directory)
